# Overview
In this notebook, we will conduct **reranking** experiment using the **msmarco_passage** dataset.  
First initialize **Pyterrier**

In [1]:
import pyterrier as pt
pt.init()


/home/djh/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


import **ColBERTModelOnlyFactory**

In [2]:
from pyterrier_colbert.ranking import ColBERTModelOnlyFactory

colbert_model_path = Checkpoint we already have or have trained

In [3]:
colbert_model_path = '/mnt/c/Users/DJH/Desktop/code/colbertv2.0'

ColBERTModelOnlyFactory(colbert_model_path, gpu = [True, False])

In [4]:
factory_v2 = ColBERTModelOnlyFactory(colbert_model_path, gpu=True)

load **msmarco_passage** dataset and **topics** using **Pyterrier**

In [5]:
br = pt.BatchRetrieve.from_dataset('msmarco_passage', 'terrier_stemmed_text', num_results=100, metadata=['docno', 'text'])

In [6]:
res_2020 = br(pt.get_dataset('msmarco_passage').get_topics('test-2020'))

In [7]:
res_2020

,qid,docid,docno,text,rank,score,query
0,1030303,8726436,8726436,Share on LinkedInShare on FacebookShare on Twi...,0,23.228384,who is aziz hashim
1,1030303,8726429,8726429,"The crew at NRD Holdings, left to right: Karim...",1,18.600225,who is aziz hashim
2,1030303,8726435,8726435,Mr. Aziz Hashim has been the President and Sec...,2,18.566148,who is aziz hashim
3,1030303,8726433,8726433,Romaniello calls Hashim a terrific leader at t...,3,18.257022,who is aziz hashim
4,1030303,8726437,8726437,Aziz Hashim is one of the worlds leading exper...,4,16.176181,who is aziz hashim
...,...,...,...,...,...,...,...
19611,132622,3946842,3946842,This Appendix lists the common & chemical name...,95,9.350961,definition of attempted arson
19612,132622,3358939,3358939,Arson is prosecuted with the attention to degr...,96,9.315411,definition of attempted arson
19613,132622,5679342,5679342,The tool you used to root may allow you to unr...,97,9.290289,definition of attempted arson
19614,132622,6877872,6877872,The dosage would be 100 mg twice a day for 7 d...,98,9.290289,definition of attempted arson


**Reranking**  
.text_scorer(**Do not** fill in the parameters to use ColBERT V2's new score algorithm )(the preprocessed dataset)

In [15]:
import time

start_time = time.time()

re_ranked_2020 = factory_v2.text_scorer()(res_2020)

end_time = time.time()

elapsed_time = end_time - start_time
print(f"V2 reranking time: {elapsed_time:.2f} seconds")

V2 reranking time: 60.05 seconds


In [10]:
print(re_ranked_2020)

           qid               query    docno      score  rank
0      1030303  who is aziz hashim  8726437  29.203125     0
1      1030303  who is aziz hashim  8726435  27.078125     1
2      1030303  who is aziz hashim  7156982  25.968750     2
3      1030303  who is aziz hashim  8726433  25.734375     3
4      1030303  who is aziz hashim  8726430  24.875000     4
...        ...                 ...      ...        ...   ...
19611   999466    where is velbert  8748919  22.453125     5
19612   999466    where is velbert  8748917  20.765625     6
19613   999466    where is velbert  8748918  20.203125     7
19614   999466    where is velbert  8748915  19.875000     8
19615   999466    where is velbert  8748913  19.828125     9

[19616 rows x 5 columns]


Evaluate the NDCG@10 metric **without** reranking

In [11]:
pt.Evaluate(res_2020, pt.get_dataset('msmarco_passage').get_qrels('test-2020'), [pt.measures.NDCG@10])

{'nDCG@10': 0.4505452139735618}

Evaluate the NDCG@10 metric **after** reranking

In [13]:
print(f"After V2 reranking")
pt.Evaluate(re_ranked_2020, pt.get_dataset('msmarco_passage').get_qrels('test-2020'), [pt.measures.NDCG@10])

After V2 reranking


{'nDCG@10': 0.6952931776901474}